#### Data analysis notebook

Reads in the data files and plot results



In [ ]:
from RunManager import RunManager
from Geant4Analyzer import Geant4Analyzer
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
manager = RunManager("../run/rundb.json")
display(manager.display_all_runs(include_deleted=False))

In [ ]:
# cut on the global event variables
gxe = 0 # gaseous xenon
lxe = 1 # liquid xenon
nai = 2 # NaI detector

cut = lambda data: (data['ndet'][:,lxe] > -1)
# additional cuts on the clusters
cut_hit = lambda data: (data['eh'] > 0.) & (data['id'] >-1)

std = Geant4Analyzer("run_03", first_only=False)
std.preprocess_data(cut=cut, cut_hit=cut_hit)


In [ ]:
# 2D plot with log color scale
h = plt.hist2d(std.data['xh'], std.data['yh'], bins=500, range=[[-200,200],[-100,300]], cmap='viridis')

In [ ]:
ax = std.plot_histogram("eh", bins=200, range=(0, 2000), show=False)

ax.set_ylim(1e-1, 1e5)
ax.set_yscale('log')
plt.show() 

In [ ]:
h2 = plt.hist2d(std.data['r'], std.data['zh'], bins=500, range=((0, 300), (-150, 50)))
plt.xlabel("r (mm)")
plt.ylabel("z (mm)")

In [ ]:
h2 = plt.hist2d(std.data['r'], std.data['zh'], bins=100, range=((0, 120), (-120, 20)))
plt.xlabel("r (mm)")
plt.ylabel("z (mm)")

In [ ]:
h = plt.hist(std.data['ndet'][:,lxe], bins=10, range=(-0.5, 9.5))
plt.yscale('log')